# 🧫 🧪 Эксперименты с классическими *unsupervised* методами обнаружения аномалий

### 🌐 Установка [pyod](https://github.com/yzhao062/pyod)

In [1]:
# !pip3 install pyod

## 💅 Предобработка данных

In [2]:
# from bs4 import BeautifulSoup
# from gensim.parsing.preprocessing import remove_stopwords
# from gensim.parsing.preprocessing import strip_short
# from gensim.parsing.preprocessing import strip_non_alphanum
# from gensim.parsing.preprocessing import strip_numeric
# from gensim.utils import tokenize
# import nltk; nltk.download('wordnet')
# from nltk.stem import WordNetLemmatizer


# def strip_html_tags(text):
#     """Удаление html tags из текста."""
#     soup = BeautifulSoup(text, "html.parser")
#     stripped_text = soup.get_text(separator=" ")
#     return stripped_text


# def get_wordnet_pos(word):
#     """Map POS tag to first character lemmatize() accepts"""
#     tag = nltk.pos_tag([word])[0][1][0].upper()
#     tag_dict = {"J": wordnet.ADJ,
#                 "N": wordnet.NOUN,
#                 "V": wordnet.VERB,
#                 "R": wordnet.ADV}
#     return tag_dict.get(tag, wordnet.NOUN)


# def preprocess_text(text):
#     text = strip_html_tags(text)  # удаление html tags
#     text = strip_non_alphanum(text) # заменили все небуквенные символы на пробел
#     text = strip_numeric(text) # удалили все цифры
#     text = remove_stopwords(text) # удалили все стоп-слова
#     # text = strip_short(text, minsize=2) # удалили короткие слова
#     word_list = list(tokenize(text, deacc=True, to_lower=True)) # токенизация, deacc - избавляет от ударений
#     word_list = [WordNetLemmatizer().lemmatize(word) for word in word_list] # лемматизация
#     return ' '.join(word for word in word_list)

## 🧬 Экперименты

In [3]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from sklearn.datasets import fetch_20newsgroups
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score

import pyod
from pyod.models import iforest
from pyod.models import lof
from pyod.models import knn
from pyod.models import copod

import json

c = 0.1  # отношение количества аномальных экземпляров к нормальным

# Загрузка предобработанных текстов
with open('my_dict.json', 'r') as fp:
    topics_dict = json.load(fp)

experimant_cnt = 0
all_experiments = len(topics_dict) * (len(topics_dict) -  1)
auc_list_iforest = []
auc_list_lof = []
auc_list_knn = []
auc_list_copod = []


# Перебираем пары категорий
for t1 in topics_dict:
    for t2 in topics_dict:
        if t1 == t2:
            continue

        experimant_cnt += 1
        
        if t1 == 'Наука и техника-Наука' or experimant_cnt in (3, 10, 17, 32, 36, 38, 43, 45, 46, 52, 53):
            continue

        # Формирование нормальной и аномальной выборок
        normal_data = topics_dict[t1]
        anomal_data = topics_dict[t2][:min(int(c * len(normal_data)) + 1, len(topics_dict[t2]))]
        all_data = normal_data + anomal_data

        # TF-IDF векторизация
        vectorizer = TfidfVectorizer()
        all_data_tf = vectorizer.fit_transform(all_data).toarray()

        # Формирование выборок
        x = all_data_tf
        y = np.array([False] * len(normal_data) + [True] * len(anomal_data))
        all_data, x, y = shuffle(all_data, x, y, random_state=123)

        # Задаем модели

        # Isolation Forest
        iforest_clf = iforest.IForest(
            contamination=0.1,
            n_estimators=3000,
            max_samples=1.0,
            random_state=123,
            n_jobs=-1,
        )

        # LOF
        lof_clf = lof.LOF(
            contamination=0.1,
            n_neighbors=3,
            metric='canberra',
            n_jobs=-1,
        )

        #kNN
        knn_clf = knn.KNN(
            contamination=0.1,
            n_neighbors=3,
            method='largest',
            metric='canberra',
            n_jobs=-1,
        )

        # COPOD
        copod_clf = copod.COPOD(contamination=0.1)

        # Считаем метрику ROC AUC
        auc_iforest = iforest_clf.fit_predict_score(x, y)
        auc_lof = lof_clf.fit_predict_score(x, y)
        auc_knn = knn_clf.fit_predict_score(x, y)
        auc_copod = copod_clf.fit_predict_score(x, y)

        # Добавляем в списки
        auc_list_iforest.append(auc_iforest)
        auc_list_lof.append(auc_lof)
        auc_list_knn.append(auc_knn)
        auc_list_copod.append(auc_copod)

        # Вывод результатов
        print("-" * 50)
        print("Эксперимент №{}/{}  с normal = {}, anomal = {}".format(
            experimant_cnt, all_experiments, t1, t2))
        print("auc_iforest = ", auc_iforest)
        print("auc_lof = ", auc_lof)
        print("auc_knn = ", auc_knn)
        print("auc_copod = ", auc_copod)
        print("-" * 50)


auc_np = np.array(auc_list_iforest)
print("*" * 50)
print("IFOREST Медиана auc = {}".format(np.median(auc_np)))
print("IFOREST Среднее auc = {}".format(np.mean(auc_np)))
print("*" * 50)

auc_np = np.array(auc_list_lof)
print("*" * 50)
print("LOF Медиана auc = {}".format(np.median(auc_np)))
print("LOF Среднее auc = {}".format(np.mean(auc_np)))
print("*" * 50)

auc_np = np.array(auc_list_knn)
print("*" * 50)
print("kNN Медиана auc = {}".format(np.median(auc_np)))
print("kNN Среднее auc = {}".format(np.mean(auc_np)))
print("*" * 50)

auc_np = np.array(auc_list_copod)
print("*" * 50)
print("COPOD Медиана auc = {}".format(np.median(auc_np)))
print("COPOD Среднее auc = {}".format(np.mean(auc_np)))
print("*" * 50)

roc_auc_score: 0.8628316831683168
roc_auc_score: 0.7677722772277228
roc_auc_score: 0.7837326732673268
roc_auc_score: 0.8372079207920792
--------------------------------------------------
Эксперимент №1/56  с normal = Наука и техника-Оружие, anomal = Культура-Театр
auc_iforest =  0.8628316831683168
auc_lof =  0.7677722772277228
auc_knn =  0.7837326732673268
auc_copod =  0.8372079207920792
--------------------------------------------------
roc_auc_score: 0.6815247524752475
roc_auc_score: 0.5334851485148515
roc_auc_score: 0.4737029702970297
roc_auc_score: 0.528990099009901
--------------------------------------------------
Эксперимент №2/56  с normal = Наука и техника-Оружие, anomal = Силовые структуры-Полиция и спецслужбы
auc_iforest =  0.6815247524752475
auc_lof =  0.5334851485148515
auc_knn =  0.4737029702970297
auc_copod =  0.528990099009901
--------------------------------------------------
roc_auc_score: 0.7106039603960397
roc_auc_score: 0.7044257425742574
roc_auc_score: 0.660539603

roc_auc_score: 0.7912376237623763
roc_auc_score: 0.8346435643564357
--------------------------------------------------
Эксперимент №22/56  с normal = Спорт-Бокс и ММА, anomal = Наука и техника-Оружие
auc_iforest =  0.9207821782178218
auc_lof =  0.7701782178217822
auc_knn =  0.7912376237623763
auc_copod =  0.8346435643564357
--------------------------------------------------
roc_auc_score: 0.9531485148514851
roc_auc_score: 0.9197623762376237
roc_auc_score: 0.961089108910891
roc_auc_score: 0.9603762376237625
--------------------------------------------------
Эксперимент №23/56  с normal = Спорт-Бокс и ММА, anomal = Культура-Театр
auc_iforest =  0.9531485148514851
auc_lof =  0.9197623762376237
auc_knn =  0.961089108910891
auc_copod =  0.9603762376237625
--------------------------------------------------
roc_auc_score: 0.8630792079207921
roc_auc_score: 0.7047623762376238
roc_auc_score: 0.7004851485148514
roc_auc_score: 0.7608415841584157
--------------------------------------------------
Э

roc_auc_score: 0.7174653465346534
roc_auc_score: 0.4849108910891089
roc_auc_score: 0.5045198019801981
roc_auc_score: 0.5855643564356435
--------------------------------------------------
Эксперимент №54/56  с normal = Экономика-Госэкономика, anomal = Наука и техника-Космос
auc_iforest =  0.7174653465346534
auc_lof =  0.4849108910891089
auc_knn =  0.5045198019801981
auc_copod =  0.5855643564356435
--------------------------------------------------
roc_auc_score: 0.9381683168316831
roc_auc_score: 0.674039603960396
roc_auc_score: 0.6590594059405941
roc_auc_score: 0.7853168316831683
--------------------------------------------------
Эксперимент №55/56  с normal = Экономика-Госэкономика, anomal = Наука и техника-Наука
auc_iforest =  0.9381683168316831
auc_lof =  0.674039603960396
auc_knn =  0.6590594059405941
auc_copod =  0.7853168316831683
--------------------------------------------------
roc_auc_score: 0.8104455445544555
roc_auc_score: 0.6128019801980198
roc_auc_score: 0.6257326732673267